<a href="https://colab.research.google.com/github/VaasuDevanS/python-notebooks/blob/main/StrataScratch/Meta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2005 Share of Active Users
https://platform.stratascratch.com/coding/2005-share-of-active-users?code_type=2

In [ ]:
result = (
    len(fb_active_users.query('status == "open" and country == "USA"')) /
    len(fb_active_users.query('country == "USA"'))
)

# 2119 Most Lucrative Products
https://platform.stratascratch.com/coding/2119-most-lucrative-products?code_type=2

In [ ]:
result = (
    online_orders
        .query('"2022-01-01" <= date <= "2022-06-30"')
        .assign(total=online_orders['units_sold'] * online_orders['cost_in_dollars'])
        .groupby('product_id', as_index=False)
        .sum()
        .nlargest(5, 'total')
        [['product_id', 'total']]
)

# 10061 Popularity of Hack
https://platform.stratascratch.com/coding/10061-popularity-of-hack?code_type=2

In [ ]:
result = (
    facebook_employees
        .merge(facebook_hack_survey, left_on='id', right_on='employee_id')
        .groupby('location', as_index=False)
        ['popularity']
        .mean()
)

# 10087 Find all posts which were reacted to with a heart
https://platform.stratascratch.com/coding/10087-find-all-posts-which-were-reacted-to-with-a-heart?code_type=2

In [ ]:
result = (
    facebook_reactions
        .merge(facebook_posts, on='post_id')
        .query('reaction == "heart"')
        [['post_id', 'poster_x', 'post_text', 'post_keywords', 'post_date']]
        .drop_duplicates()
        .rename(columns={'poster_x': 'poster'})
)

# 9781 Find the rate of processed tickets for each type
https://platform.stratascratch.com/coding/9781-find-the-rate-of-processed-tickets-for-each-type?code_type=2

In [ ]:
result = (
    facebook_complaints
        .groupby('type', as_index=False)
        ['processed']
        .mean()
)

# 10064 Highest Energy Consumption
https://platform.stratascratch.com/coding/10064-highest-energy-consumption?code_type=2

In [ ]:
result = (
    pd.concat([fb_eu_energy, fb_asia_energy, fb_na_energy])
        .groupby('date', as_index=False)
        .sum()
        .nlargest(1, 'consumption', keep='all')
)

# 10134 Spam Posts
https://platform.stratascratch.com/coding/10134-spam-posts?code_type=2

In [ ]:
result = (
    facebook_posts
        .assign(spam_share=facebook_posts['post_keywords'].str.contains('spam'))
        .merge(facebook_post_views, on='post_id')
        .groupby('post_date', as_index=False)
        ['spam_share']
        .agg(lambda x: x.mean() * 100)
)

# 10284 Popularity Percentage
https://platform.stratascratch.com/coding/10284-popularity-percentage?code_type=2

In [ ]:
result = (
    (combined := pd.concat((facebook_friends.user1, facebook_friends.user2)))
        .value_counts()
        / combined.nunique()
        * 100
)

# 10285 Acceptance Rate By Date
https://platform.stratascratch.com/coding/10285-acceptance-rate-by-date?code_type=2

In [ ]:
result = (
    pd.merge(
        fb_friend_requests.query('action == "sent"'),
        fb_friend_requests.query('action == "accepted"'),
        on=['user_id_sender', 'user_id_receiver'],
        how='left'
    )
    .assign(acceptance_rate=lambda x: x['date_y'].notnull())
    .groupby('date_x', as_index=False)
    ['acceptance_rate']
    .mean()
    .sort_values('date_x')
)

# 10352 Users By Average Session Time
https://platform.stratascratch.com/coding/10352-users-by-avg-session-time?code_type=2

In [ ]:
def filter_df(df: pd.DataFrame, action: str):
    df = df.query(f'action == "{action}"')
    df['day'] = df['timestamp'].dt.date
    return df.groupby(['user_id', 'day'], as_index=False)


df_load = filter_df(facebook_web_log, 'page_load').max()
df_exit = filter_df(facebook_web_log, 'page_exit').min()

merged = pd.merge(df_load, df_exit, on=['user_id', 'day'])
merged['duration'] = merged['timestamp_y'] - merged['timestamp_x']
merged.groupby('user_id', as_index=False)['duration'].mean()